In [14]:
import re


In [15]:
def extract_functions(file_content):
    # 正则表达式模式以匹配C/C++函数声明
    function_pattern = r'(\w+)\s+(\w+)\s*\(([^)]*)\)\s*{'
    functions = re.findall(function_pattern, file_content)
    return functions

In [16]:
def find_function_body(return_type, function_name, functions_lines):
    key = return_type + " " + function_name
    start_line = [index for index, item in enumerate(functions_lines, 0) if key in item][0]
    in_function_body = False
    brace_count = 0  # 用于跟踪花括号的嵌套层级
    for i, line in enumerate(functions_lines[start_line:]):
        if not in_function_body:
            if '{' in line:
                in_function_body = True
                brace_count = 1
        else:
            if '{' in line:
                brace_count = brace_count + 1
            if "}" in line:
                brace_count = brace_count - 1
            if brace_count == 0:
                if start_line + i < len(functions_lines):
                    return functions_lines[start_line:start_line + i + 1]
                else:
                    return functions_lines[start_line:-1]

In [17]:
def find_called_functions(function_body):
    # 正则表达式模式以匹配函数调用
    call_pattern = r'\b(\w+)\s*\('
    called_functions = re.findall(call_pattern, function_body)
    return called_functions


In [18]:
with open('CWE789_Uncontrolled_Mem_Alloc__malloc_char_fscanf_01.c', 'r') as file:
    file_content = file.read()

cleaned_code = re.sub(r'/\*.*?\*/', '', file_content, flags=re.DOTALL)  # 使用正则表达式匹配和删除多行注释
cleaned_code = re.sub(r'//.*', '', cleaned_code)  # 使用正则表达式匹配和删除单行注释

functions_lines = [item for item in cleaned_code.split('\n') if item]
functions = extract_functions(file_content)
function_dic = {}
functions

[('int', 'bad', ''),
 ('int', 'goodG2B', ''),
 ('int', 'goodB2G', ''),
 ('void', 'CWE789_Uncontrolled_Mem_Alloc__malloc_char_fscanf_01_good', ''),
 ('int', 'main', 'int argc, char * argv[]')]

In [19]:
def find_duplicates(arr):
    seen = set()
    duplicates = set()

    for item in arr:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)

    return list(duplicates)


In [20]:
functions_name_list = []

for k in functions:
    functions_name_list.append(k[1])
good_function = []


def starts_with_good(input_str):
    if input_str.startswith("good") and len(input_str) > 4:
        good_function.append(input_str)


for k in functions_name_list:
    starts_with_good(k)
good_function

['goodG2B', 'goodB2G']

In [21]:
import random

def random_choice(my_array):
    if my_array:
        return random.choice(my_array)
    else:
        return None

selected_element = random_choice(good_function)



In [22]:
def contains_bad(input_str):
    return "_bad" in input_str
bad_one=None
good_one=None
void=0

In [23]:
for return_type, function_name, function_para in functions:
    print("\n")
    print("rteurn value" + return_type)
    print("function name" + function_name)
    print("para" + function_para)
    function_body = find_function_body(return_type, function_name, functions_lines)
    print(function_body)

    
    if contains_bad(function_name) or function_name=='bad':
        bad_one=function_body
        if return_type.lower()=='void':
            void=1
            continue;
    if selected_element.lower()==function_name.lower():
        good_one=function_body
        if return_type.lower()=='void':
            void=1
            continue;






rteurn valueint
function namebad
para
['int bad()', '{', '    size_t data;', '    ', '    data = 0;', '    ', '    fscanf(stdin, "%zu", &data);', '    {', '        char * myString;', '        ', '        ', '        if (data > strlen(HELLO_STRING))', '        {', '            myString = (char *)malloc(data*sizeof(char));', '            if (myString == NULL) {exit(-1);}', '            ', '            strcpy(myString, HELLO_STRING);', '            printLine(myString);', '            free(myString);', '        }', '        else', '        {', '            printLine("Input is less than the length of the source string");', '        }', '    }', '}']


rteurn valueint
function namegoodG2B
para
['static int goodG2B()', '{', '    size_t data;', '    ', '    data = 0;', '    ', '    data = 20;', '    {', '        char * myString;', '        ', '        ', '        if (data > strlen(HELLO_STRING))', '        {', '            myString = (char *)malloc(data*sizeof(char));', '            if (mySt

In [24]:
bad_one

['int bad()',
 '{',
 '    size_t data;',
 '    ',
 '    data = 0;',
 '    ',
 '    fscanf(stdin, "%zu", &data);',
 '    {',
 '        char * myString;',
 '        ',
 '        ',
 '        if (data > strlen(HELLO_STRING))',
 '        {',
 '            myString = (char *)malloc(data*sizeof(char));',
 '            if (myString == NULL) {exit(-1);}',
 '            ',
 '            strcpy(myString, HELLO_STRING);',
 '            printLine(myString);',
 '            free(myString);',
 '        }',
 '        else',
 '        {',
 '            printLine("Input is less than the length of the source string");',
 '        }',
 '    }',
 '}']

In [25]:
good_one

['static int goodB2G()',
 '{',
 '    size_t data;',
 '    ',
 '    data = 0;',
 '    ',
 '    fscanf(stdin, "%zu", &data);',
 '    {',
 '        char * myString;',
 '        ',
 '        ',
 '        if (data > strlen(HELLO_STRING) && data < 100)',
 '        {',
 '            myString = (char *)malloc(data*sizeof(char));',
 '            if (myString == NULL) {exit(-1);}',
 '            ',
 '            strcpy(myString, HELLO_STRING);',
 '            printLine(myString);',
 '            free(myString);',
 '        }',
 '        else',
 '        {',
 '            printLine("Input is less than the length of the source string or too large");',
 '        }',
 '    }',
 '}']

In [26]:
#accessing the return type


if void == 0:
    func_num = 0
first_function_return_type = functions[0][0]
#checking the file type that needs to be created
file_extension = 'cpp' if 'cpp' in file.name else 'c'
output_file_name = f"equivalencetest.{file_extension}"

with open(output_file_name, 'w') as output_file:
    pass

if void == 0:


    #writing the main
    with open(output_file_name, 'w') as output_file:
        for k in bad_one:
            output_file.write(k + "\n")
        for k in good_one:
            output_file.write(k + '\n')
        
    
        output_file.write("\n")
        output_file.write("\nint main()\n{\n")
        output_file.write("    " + str(first_function_return_type) + " x;\n")
        output_file.write("    s2e_make_symbolic(&x, sizeof(x), \"x\");\n")

        func_calls = []
        for func_num, (return_type, function_name, function_para) in enumerate(functions):
            call_str = f"    {return_type} func{func_num} = {function_name}(x);\n"
            output_file.write(call_str)
            func_calls.append(f"func{func_num}")

        if len(func_calls) >= 2:
            # Create a comparison string that compares all function calls for equality
            comparison_string = " == ".join(func_calls)
            assert_statement = f"    s2e_assert({comparison_string});\n"
            output_file.write(assert_statement)

        output_file.write("    return 0;\n")
        output_file.write("}\n")
    print("success write")
else:
    print('the return type error')


success write
